# Pre-processing & Training Data Development

## Import Libraries

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os

## Load Data

In [2]:
dialysis = pd.read_csv('Data/capstone2_dialysis_cleaned_data.csv', index_col=0)

## Thin to Relevant Columns

In [3]:
dialysis_columns =['Network',
                   'State',
                   'Facility Name',
                   'Star Rating', 
                   'Profit Or Non-Profit', 
                   'Chain Owned',
                   'Chain Organization', 
                   'Late Shift', 
                   'Number Of Dialysis Stations',
                   'Offers In-Center Hemodialysis', 
                   'Offers Peritoneal Dialysis',
                   'Offers Home Hemodialysis Training',
                   'Percentage Of Medicare Patients With Hgb < 10 g/dL',
                   'Percentage Of Medicare Patients With Hgb > 12 g/dL',
                   'Percent Of Adult HD Patients With Kt/V >= 1.2',
                   'Percentage Of Adult Patients With Hypercalcemia (Serum Calcium Greater Than 10.2 mg/dL)',
                   'Percentage Of Adult Patients With Serum Phosphorus Less Than 3.5 mg/dL',
                   'Percentage Of Adult Patients With Serum Phosphorus Between 3.5 - 4.5 mg/dL',
                   'Percentage Of Adult Patients With Serum Phosphorus Between 4.6 - 5.5 mg/dL',
                   'Percentage Of Adult Patients With Serum Phosphorus Between 5.6 - 7.0 mg/dL',
                   'Percentage Of Adult Patients With Serum Phosphorus Greater Than 7.0 mg/dL',
                   'Mortality Rate (Facility)',
                   'Readmission Rate (Facility)',
                   'Hospitalization Rate (Facility)',
                   'Standard Infection Ratio',
                   'Transfusion Rate (Facility)',
                   'Fistula Rate (Facility)',
                   'Percentage Of Adult Patients With Long Term Catheter In Use',
                   'Standardized First Kidney Transplant Waitlist Ratio',
                   'Percentage Of Prevalent Patients Waitlisted']

In [4]:
dialysis_thin = dialysis[dialysis_columns]

## Dummy Variables

In [5]:
# Select DataFrame object columns
dialysis_thin.select_dtypes('object')

,State,Facility Name,Chain Organization
0,AZ,FMC NORTH MOUNTAIN,FRESENIUS MEDICAL CARE
1,AR,DAVITA POCAHONTAS DIALYSIS,DAVITA
2,CO,DAVITA PLATTE VALLEY DIALYSIS,DAVITA
3,FL,DAVITA MELBOURNE DIALYSIS,DAVITA
4,GA,DAVITA WEST HIRAM DIALYSIS,DAVITA
...,...,...,...
7719,TN,DAVITA SUMNER DIALYSIS,DAVITA
7720,WI,FRESENIUS MEDICAL CARE OSHKOSH LLC,FRESENIUS MEDICAL CARE
7721,CA,DAVITA CIRCLE CITY DIALYSIS,DAVITA
7722,TX,"FRESENIUS MEDICAL CARE RENNER ROAD, LLC",FRESENIUS MEDICAL CARE


#### State

In [6]:
# Use one-hot encoder to create dummy variables for State column
# Drop_first=True to avoid redundancy
dialysis_dumb = pd.get_dummies(dialysis_thin, columns=['State'], prefix='State', drop_first=True)

#### Chain Organization

In [7]:
# Relabel chain organization values that only occur once 'Other'
counts = dialysis_dumb['Chain Organization'].value_counts()
mask = dialysis_dumb['Chain Organization'].isin(counts[counts==1].index)
dialysis_dumb['Chain Organization'][mask] = 'Other'

/Users/nicholehartz/opt/anaconda3/envs/TestEnv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
# Verify chain organziation values that only occur once re-labelled 'Other'
dialysis_dumb['Chain Organization'].value_counts()

DAVITA                                     2908
FRESENIUS MEDICAL CARE                     2674
INDEPENDENT                                 818
US RENAL CARE, INC.                         283
DIALYSIS CLINIC, INC.                       258
AMERICAN RENAL ASSOCIATES                   245
SATELLITE HEALTHCARE                         64
Other                                        60
DIVERSIFIED SPECIALTY INSTITUTES (DSI)       22
RENAL VENTURES MANAGEMENT                    20
CENTERS FOR DIALYSIS CARE                    19
KAISER PERMANENTE                            19
STATE OWNED                                  19
NORTHWEST KIDNEY CENTERS                     19
UNIVERSITY OF UTAH DIALYSIS PROGRAM          18
WAKE FOREST UNIVERSITY                       18
ATLANTIS HEALTHCARE GROUP                    17
RENAL CARE GROUP INC.                        16
GREENFIELD HEALTH SYSTEMS                    15
SATELLITE DIALYSIS                           15
SANKAR NEPHROLOGY GROUP                 

In [9]:
# Use one-hot encoder to create dummy variables for Chain Organization column
# Drop_first=True to avoid redundancy
dialysis_dumb = pd.get_dummies(dialysis_dumb, columns=['Chain Organization'], prefix='Chain Organization', drop_first=True)

#### Drop Remaining Categorical Columns

In [10]:
# Drop Facility Name column
dialysis_dumb = dialysis_dumb.drop(['Facility Name'], axis=1)

## Standardize & Remove Outliers

In [11]:
# Initialize StandardScaler
scaler = StandardScaler()

#### Mortality Rate (Facility) 

In [12]:
# Standardize Mortality Rate (Facility) 
scaler = StandardScaler()
scaler.fit(dialysis_dumb[['Mortality Rate (Facility)']])
dialysis_dumb['Mortality Rate Standardized'] = scaler.transform(dialysis_dumb[['Mortality Rate (Facility)']])

In [13]:
# Remove outliers (values more than 3 standard deviations from the mean)
mean_mort = dialysis_dumb['Mortality Rate Standardized'].mean()
std_mort = dialysis_dumb['Mortality Rate Standardized'].std()
cut_off_mort = std_mort * 3
lower_mort, upper_mort = mean_mort - cut_off_mort, mean_mort + cut_off_mort
dialysis_scaled = dialysis_dumb[(dialysis_dumb['Mortality Rate Standardized'] < upper_mort) & (dialysis_dumb['Mortality Rate Standardized'] > lower_mort)]

#### Readmission Rate (Facility)

In [14]:
# Standardize Readmission Rate (Facility)
scaler.fit(dialysis_dumb[['Readmission Rate (Facility)']])
dialysis_dumb['Readmission Rate Standardized'] = scaler.transform(dialysis_dumb[['Readmission Rate (Facility)']])

In [15]:
# Remove outliers (values more than 3 standard deviations from the mean)
mean_read = dialysis_dumb['Readmission Rate Standardized'].mean()
std_read = dialysis_dumb['Readmission Rate Standardized'].std()
cut_off_read = std_read * 3
lower_read, upper_read = mean_read - cut_off_read, mean_read + cut_off_read
dialysis_scaled = dialysis_dumb[(dialysis_dumb['Readmission Rate Standardized'] < upper_read) & (dialysis_dumb['Readmission Rate Standardized'] > lower_read)]

#### Hospitalization Rate (Facility)

In [16]:
# Standardize Hospitalization Rate (Facility)
scaler.fit(dialysis_dumb[['Hospitalization Rate (Facility)']])
dialysis_dumb['Hospitalization Rate Standardized'] = scaler.transform(dialysis_dumb[['Hospitalization Rate (Facility)']])

In [17]:
# Remove outliers (values more than 3 standard deviations from the mean)
mean_hosp = dialysis_dumb['Hospitalization Rate Standardized'].mean()
std_hosp = dialysis_dumb['Hospitalization Rate Standardized'].std()
cut_off_hosp = std_hosp * 3
lower_hosp, upper_hosp = mean_hosp - cut_off_hosp, mean_hosp + cut_off_hosp
dialysis_scaled = dialysis_dumb[(dialysis_dumb['Hospitalization Rate Standardized'] < upper_hosp) & (dialysis_dumb['Hospitalization Rate Standardized'] > lower_hosp)]

#### Standard Infection Ratio

In [18]:
# Standardize Standard Infection Ratio
scaler.fit(dialysis_dumb[['Standard Infection Ratio']])
dialysis_dumb['Standard Infection Ratio Standardized'] = scaler.transform(dialysis_dumb[['Standard Infection Ratio']])

In [19]:
# Remove outliers (values more than 3 standard deviations from the mean)
mean_inf = dialysis_dumb['Standard Infection Ratio Standardized'].mean()
std_inf = dialysis_dumb['Standard Infection Ratio Standardized'].std()
cut_off_inf = std_inf * 3
lower_inf, upper_inf = mean_inf - cut_off_inf, mean_inf + cut_off_inf
dialysis_scaled = dialysis_dumb[(dialysis_dumb['Standard Infection Ratio Standardized'] < upper_inf) & (dialysis_dumb['Standard Infection Ratio Standardized'] > lower_inf)]

#### Transfusion Rate (Facility)

In [20]:
# Standardize Transfusion Rate (Facility)
scaler.fit(dialysis_dumb[['Transfusion Rate (Facility)']])
dialysis_dumb['Transfusion Rate Standardized'] = scaler.transform(dialysis_dumb[['Transfusion Rate (Facility)']])

In [21]:
# Remove outliers (values more than 3 standard deviations from the mean)
mean_trans = dialysis_dumb['Transfusion Rate Standardized'].mean()
std_trans = dialysis_dumb['Transfusion Rate Standardized'].std()
cut_off_trans = std_trans * 3
lower_trans, upper_trans = mean_trans - cut_off_trans, mean_trans + cut_off_trans
dialysis_scaled = dialysis_dumb[(dialysis_dumb['Transfusion Rate Standardized'] < upper_trans) & (dialysis_dumb['Transfusion Rate Standardized'] > lower_trans)]

#### Fistula Rate (Facility)

In [22]:
# Standardize Fistula Rate (Facility)
scaler.fit(dialysis_dumb[['Fistula Rate (Facility)']])
dialysis_dumb['Fistula Rate Standardized'] = scaler.transform(dialysis_dumb[['Fistula Rate (Facility)']])

In [23]:
# Remove outliers (values more than 3 standard deviations from the mean)
mean_fist = dialysis_dumb['Fistula Rate Standardized'].mean()
std_fist = dialysis_dumb['Fistula Rate Standardized'].std()
cut_off_fist = std_fist * 3
lower_fist, upper_fist = mean_fist - cut_off_fist, mean_fist + cut_off_fist
dialysis_scaled = dialysis_dumb[(dialysis_dumb['Fistula Rate Standardized'] < upper_fist) & (dialysis_dumb['Fistula Rate Standardized'] > lower_fist)]

#### Remove original, non-scaled columns from scaled DataFrame

In [24]:
dialysis_scaled = dialysis_scaled.drop(['Mortality Rate (Facility)', 'Readmission Rate (Facility)', 'Hospitalization Rate (Facility)', 'Standard Infection Ratio', 'Transfusion Rate (Facility)', 'Fistula Rate (Facility)'], axis=1)

## Remove remaining outliers

In [25]:
percent = ['Percentage Of Medicare Patients With Hgb < 10 g/dL',
            'Percentage Of Medicare Patients With Hgb > 12 g/dL',
            'Percent Of Adult HD Patients With Kt/V >= 1.2',
            'Percentage Of Adult Patients With Hypercalcemia (Serum Calcium Greater Than 10.2 mg/dL)',
            'Percentage Of Adult Patients With Serum Phosphorus Less Than 3.5 mg/dL',
            'Percentage Of Adult Patients With Serum Phosphorus Between 3.5 - 4.5 mg/dL',
            'Percentage Of Adult Patients With Serum Phosphorus Between 4.6 - 5.5 mg/dL',
            'Percentage Of Adult Patients With Serum Phosphorus Between 5.6 - 7.0 mg/dL',
            'Percentage Of Adult Patients With Serum Phosphorus Greater Than 7.0 mg/dL',
            'Percentage Of Adult Patients With Long Term Catheter In Use',
            'Percentage Of Prevalent Patients Waitlisted']

In [26]:
for col in dialysis[percent].columns:
    mean = dialysis[col].mean()
    std = dialysis[col].std()
    cut_off = std * 3
    lower, upper = mean - cut_off, mean + cut_off
    dialysis = dialysis[(dialysis[col] < upper) & (dialysis[col] > lower)]

## Train / Test Split

In [27]:
X = dialysis_scaled.drop(['Star Rating'], axis=1)
y = dialysis_scaled['Star Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Export to new csv

In [28]:
os.getcwd()

'/Users/nicholehartz/Documents/DataScience/Projects/Capstone2'

In [29]:
dialysis_thin.to_csv('Data/capstone2_dialysis_thin_data.csv')

In [30]:
dialysis_dumb.to_csv('Data/capstone2_dialysis_dumb_data.csv')

In [31]:
dialysis_scaled.to_csv('Data/capstone2_dialysis_scaled_data.csv')